<a href="https://colab.research.google.com/github/shrys43/Att/blob/main/Mypro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install face_recognition
!pip install tensorflow[gpu]

In [ ]:
!pip install dlib

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import os
import cv2

# Define the base directory for attendance data
attendance_data_dir = 'attendance_data'
os.makedirs(attendance_data_dir, exist_ok=True)

# Function to collect face data for a given person
def collect_face_data(person_name, num_images=20):
    person_dir = os.path.join(attendance_data_dir, person_name)
    os.makedirs(person_dir, exist_ok=True)

    # Initialize webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    print(f"Collecting face data for {person_name}. Look at the camera and press 'c' to capture images.")

    img_count = 0
    while img_count < num_images:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break

        cv2.imshow('Collect Face Data', frame)

        # Capture image on pressing 'c'
        if cv2.waitKey(1) & 0xFF == ord('c'):
            img_name = f"{person_name}_{img_count+1:02d}.jpg"
            img_path = os.path.join(person_dir, img_name)
            cv2.imwrite(img_path, frame)
            print(f"Captured {img_name}")
            img_count += 1

        # Break the loop on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Collection stopped by user.")
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Finished collecting {img_count} images for {person_name}.")

# Example usage: Replace 'person_name' with the actual name of the person
# collect_face_data('person_name')

In [ ]:
import face_recognition
import cv2
import numpy as np
import os

def align_and_normalize_faces(input_dir, output_dir, desired_size=(160, 160)):
    """
    Aligns and normalizes faces in images from the input directory and saves them
    to the output directory.

    Args:
        input_dir (str): Directory containing the collected face images.
        output_dir (str): Directory to save the processed images.
        desired_size (tuple): The desired size of the normalized face images (width, height).
    """
    os.makedirs(output_dir, exist_ok=True)

    for person_name in os.listdir(input_dir):
        person_input_dir = os.path.join(input_dir, person_name)
        person_output_dir = os.path.join(output_dir, person_name)
        os.makedirs(person_output_dir, exist_ok=True)

        if not os.path.isdir(person_input_dir):
            continue

        print(f"Processing images for {person_name}...")

        for image_name in os.listdir(person_input_dir):
            image_path = os.path.join(person_input_dir, image_name)
            output_image_path = os.path.join(person_output_dir, image_name)

            try:
                # Load the image
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Warning: Could not read image {image_path}")
                    continue

                # Find face locations and landmarks
                face_locations = face_recognition.face_locations(image)
                face_landmarks_list = face_recognition.face_landmarks(image, face_locations)

                if not face_landmarks_list:
                    print(f"Warning: No faces found in {image_path}")
                    continue

                # Assume only one face per image for simplicity
                face_landmarks = face_landmarks_list[0]

                # Get eye coordinates (average of left and right eye points)
                left_eye = np.mean(face_landmarks['left_eye'], axis=0).astype(int)
                right_eye = np.mean(face_landmarks['right_eye'], axis=0).astype(int)

                # Calculate the angle to rotate to align eyes horizontally
                dy = right_eye[1] - left_eye[1]
                dx = right_eye[0] - left_eye[0]
                angle = np.degrees(np.arctan2(dy, dx))

                # Get the center of the eyes
                eyes_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)

                # Get the rotation matrix
                M = cv2.getRotationMatrix2D(eyes_center, angle, 1.0)

                # Perform the rotation
                aligned_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)

                # Find the face location in the aligned image to crop
                # Need to recalculate face locations and landmarks on the aligned image
                aligned_face_locations = face_recognition.face_locations(aligned_image)

                if not aligned_face_locations:
                    print(f"Warning: No faces found after alignment in {image_path}")
                    continue

                # Assume the largest face is the target face
                largest_face_location = max(aligned_face_locations, key=lambda loc: (loc[2] - loc[0]) * (loc[1] - loc[3]))
                top, right, bottom, left = largest_face_location

                # Crop the aligned face
                cropped_face = aligned_image[top:bottom, left:right]

                # Normalize the cropped face (resize and potentially normalize pixel values)
                normalized_face = cv2.resize(cropped_face, desired_size, interpolation=cv2.INTER_AREA)

                # Optional: Normalize pixel values (e.g., to 0-1 range)
                # normalized_face = normalized_face.astype("float32") / 255.0

                # Save the processed image
                cv2.imwrite(output_image_path, normalized_face)
                # print(f"Processed and saved {output_image_path}") # Uncomment for detailed output

            except Exception as e:
                print(f"Error processing {image_path}: {e}")

    print("Face alignment and normalization complete.")

# Define input and output directories
input_data_dir = 'attendance_data' # Assuming this is where the collected data is
processed_data_dir = 'processed_attendance_data'

# Run the alignment and normalization process
# align_and_normalize_faces(input_data_dir, processed_data_dir)

In [ ]:
# Run the alignment and normalization process
input_data_dir = 'attendance_data' # Assuming this is where the collected data is
processed_data_dir = 'processed_attendance_data'
align_and_normalize_faces(input_data_dir, processed_data_dir)

In [ ]:
import face_recognition
import os
import pickle

In [ ]:
# Define the input directory where the processed face images are stored
processed_data_dir = 'processed_attendance_data'

# Define an output path for storing the generated embeddings
embeddings_output_path = 'face_embeddings.pkl'

# Initialize an empty dictionary to store the embeddings
face_embeddings = {}

In [ ]:
# Iterate through each person's directory in the input directory
for person_name in os.listdir(processed_data_dir):
    person_dir = os.path.join(processed_data_dir, person_name)

    # Skip if it's not a directory
    if not os.path.isdir(person_dir):
        continue

    print(f"Generating embeddings for {person_name}...")

    # Initialize a list for the current person's embeddings
    face_embeddings[person_name] = []

    # Iterate through each image in a person's directory
    for image_name in os.listdir(person_dir):
        image_path = os.path.join(person_dir, image_name)

        try:
            # Load the image
            image = face_recognition.load_image_file(image_path)

            # Generate face embeddings for the loaded image
            embeddings = face_recognition.face_encodings(image)

            # Check if any embeddings were found
            if not embeddings:
                print(f"Warning: No face embeddings found in {image_path}. Skipping.")
                continue

            # Append the first embedding (assuming one face per image) to the list
            face_embeddings[person_name].append(embeddings[0])

        except Exception as e:
            print(f"Error processing image {image_path}: {e}")

print("Finished generating embeddings for all persons.")

In [ ]:
# Save the embeddings dictionary to the specified output path using pickle
with open(embeddings_output_path, 'wb') as f:
    pickle.dump(face_embeddings, f)

# Print a confirmation message
print(f"Face embeddings generated and saved to {embeddings_output_path}")

In [ ]:
import pickle
from sklearn.svm import SVC

# 1. Load the face embeddings
embeddings_file = 'face_embeddings.pkl'
with open(embeddings_file, 'rb') as f:
    face_embeddings = pickle.load(f)

# 2. Prepare data for training
X = []  # List to hold embeddings
y = []  # List to hold corresponding labels (identities)
person_names = [] # List to hold unique person names

for person_name, embeddings_list in face_embeddings.items():
    person_names.append(person_name)
    for embedding in embeddings_list:
        X.append(embedding)
        y.append(person_name)

# 3. Instantiate the SVC classifier
# Using a linear kernel for simplicity
classifier = SVC(kernel='linear', probability=True) # probability=True is needed for prediction later

# 4. Train the classifier
print("Training the classifier...")
classifier.fit(X, y)
print("Classifier training complete.")

# 5. Store the trained classifier and person names
output_classifier_file = 'trained_classifier.pkl'
output_names_file = 'person_names.pkl'

with open(output_classifier_file, 'wb') as f:
    pickle.dump(classifier, f)

with open(output_names_file, 'wb') as f:
    pickle.dump(person_names, f)

print(f"Trained classifier saved to {output_classifier_file}")
print(f"Person names saved to {output_names_file}")

In [ ]:
import cv2
import face_recognition
import pickle
import numpy as np

# Load the trained classifier and person names
try:
    with open('trained_classifier.pkl', 'rb') as f:
        classifier = pickle.load(f)
    with open('person_names.pkl', 'rb') as f:
        person_names = pickle.load(f)
except FileNotFoundError:
    print("Error: trained_classifier.pkl or person_names.pkl not found. Please train the classifier first.")
    exit()

# Initialize webcam
video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("Webcam initialized. Press 'q' to quit.")

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the face locations and face embeddings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_embeddings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Iterate through each face found in the frame
    for (top, right, bottom, left), face_embedding in zip(face_locations, face_embeddings):
        # Use the trained classifier to predict the person's identity
        if face_embedding.size > 0:
            predictions = classifier.predict([face_embedding])
            predicted_name = predictions[0]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, predicted_name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        else:
            # Handle cases where no embedding was generated for a detected face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, "Unknown", (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import face_recognition
import pickle
import numpy as np
import time
from datetime import datetime

# Load the trained classifier and person names
try:
    with open('trained_classifier.pkl', 'rb') as f:
        classifier = pickle.load(f)
    with open('person_names.pkl', 'rb') as f:
        person_names = pickle.load(f)
except FileNotFoundError:
    print("Error: trained_classifier.pkl or person_names.pkl not found. Please train the classifier first.")
    exit()

# Initialize webcam
video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("Webcam initialized. Press 'q' to quit.")

# Attendance logging variables
attendance_log = {}  # Dictionary to store attendance: {person_name: timestamp}
recognition_threshold = 5  # Time in seconds to wait before logging the same person again

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the face locations and face embeddings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_embeddings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Iterate through each face found in the frame
    for (top, right, bottom, left), face_embedding in zip(face_locations, face_embeddings):
        # Use the trained classifier to predict the person's identity
        if face_embedding.size > 0:
            predictions = classifier.predict([face_embedding])
            predicted_name = predictions[0]
            confidence = classifier.predict_proba([face_embedding])[0][np.argmax(predictions)] # Get confidence

            # Check if the predicted name is known
            if predicted_name in person_names:
                current_time = time.time()

                # Check if the person has been logged recently
                if predicted_name not in attendance_log or (current_time - attendance_log[predicted_name]) > recognition_threshold:
                    # Log attendance
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    attendance_log[predicted_name] = current_time # Update last logged time
                    print(f"Attendance logged for {predicted_name} at {timestamp}")
                    # You can also save this to a file here if needed

                # Draw a box around the face and display name
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, predicted_name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            else:
                # Handle unknown faces
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, "Unknown", (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        else:
            # Handle cases where no embedding was generated for a detected face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, "Unknown", (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()